# Consulta Postgres

In [1]:
#!pip install pandas
#!pip install nltk
#!pip install psycopg2
#!pip install sqlalchemy

In [2]:
# imports iniciais
import os
import nltk
import pandas as pd
import pandas.io.sql as psql
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [3]:
#nltk.download()

# Assunto

In [4]:
df = pd.read_csv('dados_teste.csv',delimiter='|')

In [5]:
df.describe()

,numero
count,400.000000
mean,4721.890000
std,2818.420545
min,24.000000
25%,2219.500000
50%,4693.500000
75%,7108.750000
max,9995.000000


In [6]:
df.count()

nome        400
empresa     400
endereco    400
data1       400
data2       400
numero      400
dtype: int64

In [25]:
df.head()

,nome,empresa,endereco,data1,data2,numero
0,Anjolie Q. Campbell,Luctus Company,"P.O. Box 974, 4866 Integer Av.",2018-10-23,2018-11-03,5796
1,Deanna F. Burnett,Quisque Nonummy Consulting,504-1917 Urna Avenue,2018-12-01,2020-06-10,7320
2,Cole Z. Travis,Malesuada Fames Ac Associates,"P.O. Box 627, 3320 Aliquam, Road",2020-08-15,2020-09-02,7639
3,Sonya Y. Mckinney,Mauris Non LLP,Ap #935-8507 Pede Rd.,2018-10-13,2019-12-04,82
4,Logan V. Holmes,Leo Cras Foundation,Ap #235-1628 Consectetuer Rd.,2019-07-18,2019-02-03,6218


In [89]:
import string
from nltk.corpus import stopwords
portuguese_stops = set(stopwords.words('portuguese'))
ignore = ['@', '.', '!','?',',','$','-','\'s','g','(',')','[',']','``',':','http','html','//members']
#from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
#from nltk.stem import RegexpStemmer
from nltk.stem import SnowballStemmer
# Cria o Stemmer
stemmer = SnowballStemmer('portuguese')
def preprocess_text(content):
    content.translate(str.maketrans('', '', string.punctuation))
    word_tokens = nltk.word_tokenize(content)
    #print(len(word_tokens),word_tokens)
    word_tokens2 = [w.lower() for w in word_tokens if w not in portuguese_stops]
    #print(len(word_tokens2),word_tokens2)
    word_tokens3 = [w for w in word_tokens2 if w not in ignore]
    #print(len(word_tokens3),word_tokens3)
    word_tokens4 = [stemmer.stem(w) for w in word_tokens3]
    #print(len(word_tokens4),word_tokens4)
    #return word_tokens4
    return ' '.join(word_tokens4)

In [68]:
teste = 'Teste de sentença para tokenizar'

In [69]:
preprocess_text(teste)

['test', 'sentenc', 'tokeniz']

In [75]:
df['nome'] = df['nome'].apply(preprocess_text)

In [76]:
#!pip install gensim
#!pip install sklearn
#!pip install jieba

In [77]:
from gensim import corpora, models, similarities

In [78]:
texts = df['nome']
print(texts)

0      [anjoli, q., campbell]
1        [deann, f., burnett]
2             [col, z., trav]
3        [sony, y., mckinney]
4           [logan, v., holm]
                ...          
395      [olga, i., thornton]
396           [xen, k., stok]
397      [cassidy, k., potts]
398          [kyle, p., robl]
399        [jarrod, n., robl]
Name: nome, Length: 400, dtype: object


In [79]:
dictionary = corpora.Dictionary(texts)

In [80]:
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus) 

In [83]:
keyword = preprocess_text('Vanna da silva')

In [84]:
kw_vector = dictionary.doc2bow(keyword)
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
sim = index[tfidf[kw_vector]]
for i in range(len(sim)):
    print('keyword is similar to text%d: %.4f' % (i + 1, sim[i]))

keyword is similar to text1: 0.0000
keyword is similar to text2: 0.0000
keyword is similar to text3: 0.0000
keyword is similar to text4: 0.0000
keyword is similar to text5: 0.0000
keyword is similar to text6: 0.0000
keyword is similar to text7: 0.0000
keyword is similar to text8: 0.0000
keyword is similar to text9: 0.0000
keyword is similar to text10: 0.0000
keyword is similar to text11: 0.0000
keyword is similar to text12: 0.0000
keyword is similar to text13: 0.0000
keyword is similar to text14: 0.0000
keyword is similar to text15: 0.0000
keyword is similar to text16: 0.0000
keyword is similar to text17: 0.0000
keyword is similar to text18: 0.0000
keyword is similar to text19: 0.0000
keyword is similar to text20: 0.0000
keyword is similar to text21: 0.0000
keyword is similar to text22: 0.0000
keyword is similar to text23: 0.0000
keyword is similar to text24: 0.0000
keyword is similar to text25: 0.0000
keyword is similar to text26: 0.0000
keyword is similar to text27: 0.0000
keyword is

In [86]:
from gensim import corpora, models, similarities
import jieba
texts = ['I love reading Japanese novels. My favorite Japanese writer is Tanizaki Junichiro.', 'Natsume Soseki is a well-known Japanese novelist and his Kokoro is a masterpiece.', 'American modern poetry is good. ']
keyword = 'Japan has some great novelists. Who is your favorite Japanese writer?'
texts = [jieba.lcut(text) for text in texts]
dictionary = corpora.Dictionary(texts)
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus) 
kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
sim = index[tfidf[kw_vector]]
for i in range(len(sim)):
    print('keyword is similar to text%d: %.2f' % (i + 1, sim[i]))

keyword is similar to text1: 0.50
keyword is similar to text2: 0.02
keyword is similar to text3: 0.00


In [21]:
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [87]:
from gensim import corpora, models, similarities
import jieba
texts = ['I love reading Japanese novels. My favorite Japanese writer is Tanizaki Junichiro.', 'Natsume Soseki is a well-known Japanese novelist and his Kokoro is a masterpiece.', 'American modern poetry is good. ']
keyword = 'Japan has some great novelists. Who is your favorite Japanese writer?'
texts = [jieba.lcut(text) for text in texts]
dictionary = corpora.Dictionary(texts)
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus) 
kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
sim = index[tfidf[kw_vector]]
for i in range(len(sim)):
    print('keyword is similar to text%d: %.2f' % (i + 1, sim[i]))

keyword is similar to text1: 0.50
keyword is similar to text2: 0.02
keyword is similar to text3: 0.00


In [88]:
import nltk 
import re
import math

import pandas as pd

from collections import Counter
from unicodedata import normalize
from nltk import ngrams

#Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
#Numero de tokens em sequencia
N_GRAM_TOKEN = 3

#Faz a normalizacao do texto removendo espacos a mais e tirando acentos
def text_normalizer(src):
    return re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()

#Faz o calculo de similaridade baseada no coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

#Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))

    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)

    return Counter(accumulator)

#Obtem a similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

#Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

if __name__ == "__main__":
    w1 = 'COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS'
    words = [
        'ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA', 
        'ADEGA DOS TRES IMPORTADORA', 
        'BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA', 
        'ALL WINE IMPORTADORA'
    ]

    print('Busca: ' + w1)

    #Nivel de aceite (40%)
    cutoff = 0.40
    #Sentenças similares
    result = []

    for w2 in words:
        print('\nCosine Sentence --- ' + w2)

        #Calculo usando similaridade do coseno com apenas tokens
        similarity_sentence = get_sentence_similarity(w1, w2)
        print('\tSimilarity sentence: ' + str(similarity_sentence))

        #Calculo usando similaridade do coseno com tokens e com ngramas do texto
        similarity_sentence_text_bigram = get_sentence_similarity(w1, w2, use_text_bigram=True)
        print('\tSimilarity sentence: ' + str(similarity_sentence_text_bigram))

        if similarity_sentence >= cutoff:
            result.append((w2, similarity_sentence))

    print('\nResultado:')
    #Exibe resultados
    for data in result:
        print(data)

Busca: COMERCIAL CASA DOS FRIOS - USAR LICINIO DIAS

Cosine Sentence --- ARES DOS ANDES - EXPORTACAO & IMPORTACAO LTDA
	Similarity sentence: 0.08362420100070908
	Similarity sentence: 0.26518576139191

Cosine Sentence --- ADEGA DOS TRES IMPORTADORA
	Similarity sentence: 0.10482848367219183
	Similarity sentence: 0.223606797749979

Cosine Sentence --- BODEGAS DE LOS ANDES COMERCIO DE VINHOS LTDA
	Similarity sentence: 0.0
	Similarity sentence: 0.39317854974639244

Cosine Sentence --- ALL WINE IMPORTADORA
	Similarity sentence: 0.0
	Similarity sentence: 0.09245003270420486

Resultado:


## Calculando a frequência

Calcula quantas vezes uma palavra aparece em um texto ou corpus

In [ ]:
# Vamos primeiro 
fdist = nltk.FreqDist(word_tokens)
print('\nContagem do número máximo de ocorrências do token "',fdist.max(),'" : ', fdist[fdist.max()])
print('\nNúmero total de tokens distintos : ', fdist.N())
print('\nA seguir estão os 10 tokens mais comuns')
print(fdist.most_common(40))
print("\n")

## Stop Words

Stopwords são palavras comuns que normalmente não contribuem para o significado de uma frase e que por isso podem ser ignoradas em processamento PLN. São palavras como "The" e "a" ((em inglês) ou "O/A" e "Um/Uma" ((em português). 

Muitos mecanismos de busca filtram estas palavras (stopwords), como forma de economizar espaço em seus índices de pesquisa.

Veja que aqui temos menos palavras, pois as stopwords foram removidas 

In [ ]:
# Podemos também criar uma lista de caracteres palavras que devem ser ignoradas


Veja que conseguimor limpar ainda mais as palavras a serem trabalhadas

# Part-of-Speech Tagging

O POS Tagging é o processo de rotulação de elementos textuais - tipicamente palavras e pontuação - com o fim de evidenciar 
a estrutura gramatical de um determinado trecho de texto. 

In [ ]:
from nltk.tag import pos_tag
tags = pos_tag(word_tokens3)
print(tags)

In [ ]:
# Para saber o que é cada código, utilize o exemplo abaixo (no caso para VB)
nltk.help.upenn_tagset('VB')

# Stemming

Stemming é a técnica de remover sufixos e prefixos de uma palavra, chamada stem. 

Por exemplo, o stem da palavra cooking é cook. Um bom algoritmo sabe que "ing" é um sufixo e pode ser removido. 

Stemming é muito usado em mecanismos de buscas para indexação de palavras. 

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import RegexpStemmer
from nltk.stem import SnowballStemmer
# Cria o Stemmer
stemmer = SnowballStemmer('english')
# Para português: SnowballStemmer('portuguese')
for word in word_tokens3:
    print(stemmer.stem(word))

Veja que algumas palavras aparece apenas o stem

# Lemmatization

Lemmatização na linguística, é o processo de agrupar as diferentes formas flexionadas de uma palavra para que possam ser analisadas como um único item.
Na linguística computacional, a Lemmatização é o processo algorítmico de determinação do lema para uma determinada palavra. 

A Lemmatização está intimamente relacionada com o Stemming. 

A diferença é que um stemmer opera em uma única palavra sem conhecimento do contexto e, portanto, não pode discriminar entre palavras que têm diferentes significados, dependendo da parte da fala. No entanto, os stemmers são geralmente mais fáceis de implementar e executar mais rapidamente, e a precisão reduzida pode não ser importante para algumas aplicações.

O Stemmning pode gerar palavras geralmente inexistentes, enquanto as lemas são palavras reais.

In [ ]:
from nltk.stem import WordNetLemmatizer
# Vamos aplicar às palavras
wordnet_lemmatizer = WordNetLemmatizer()
for word in word_tokens3:
    print(wordnet_lemmatizer.lemmatize(word))

Veja a diferença do Lemmatization para o Stemming

# Exercício 1

## Crie um código Python que faça o seguinte:
    
 1) Reuna em duas variáveis de texto (textoPos, textoNeg) todas as palavras de todos os documentos da categoria 'pos' e 'neg' respectivamente

 2) Execute os itens abaixo para textoPos e textoNeg
 
 2.1) Aplique o método de tokenização para separar as palavras
 
 2.2) Remova as stopwords e demais palavras que você julgar necessário para a limpeza do texto
 
 2.3) Aplique a lematização
 
 2.4) Calcule e imprima a frequência das 1000 palavras mais frequentes após todo o processamento 
 
Salve o Jupyter Notebook com os resultados e entregue pelo iLang